### Run the module

In [1]:
!python covid-19-official-data-ro.py

current date: 2020-04-02, table rows: 44
current date: 2020-04-03, table rows: 44
current date: 2020-04-04, table rows: 45
current date: 2020-04-05, table rows: 44
current date: 2020-04-06, table rows: 44
current date: 2020-04-07, table rows: 44
current date: 2020-04-08, table rows: 44
current date: 2020-04-09, table rows: 44
current date: 2020-04-10, table rows: 44
current date: 2020-04-11, table rows: 44
current date: 2020-04-12, table rows: 44
current date: 2020-04-13, table rows: 44
current date: 2020-04-14, table rows: 44
current date: 2020-04-15, table rows: 44
current date: 2020-04-16, table rows: 44
current date: 2020-04-17, table rows: 44
current date: 2020-04-18, table rows: 44
current date: 2020-04-19, table rows: 44
current date: 2020-04-20, table rows: 44
current date: 2020-04-21, table rows: 44
current date: 2020-04-22, table rows: 44
current date: 2020-04-23, table rows: 45
current date: 2020-04-24, table rows: 45
current date: 2020-04-25, table rows: 45
current date: 20

Traceback (most recent call last):
  File "covid-19-official-data-ro.py", line 130, in <module>
    main()
  File "covid-19-official-data-ro.py", line 125, in main
    all_data_df, country_data_df = data_cleaning(all_data_df, country_data_df)
  File "covid-19-official-data-ro.py", line 105, in data_cleaning
    country_data_df[feature] = country_data_df[feature].astype(int)
  File "C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py", line 5882, in astype
    dtype=dtype, copy=copy, errors=errors, **kwargs
  File "C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\internals\managers.py", line 581, in astype
    return self.apply("astype", dtype=dtype, **kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\internals\managers.py", line 438, in apply
    applied = getattr(b, f)(**kwargs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\internals\blocks.py", line 559, in astype
    return self._astype(dtype, copy=copy, errors=errors, values